In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib
import requests
# from PyPDF2 import PdfMerger
from urllib.parse import urlparse, urlunparse
from io import BytesIO
from enum import Enum
import re
from urllib.error import URLError, HTTPError  # Import these classes from urllib.error
import ssl
import pymupdf
import os
import undetected_chromedriver as uc


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import tempfile
import time
import xml.etree.ElementTree as ET
import grobid_tei_xml
import json

## Extract Link from 150 Good research paper

In [2]:
# Read in the csv file
## This is a csv file that teammates manually inspected as good research pape
good_df = pd.read_csv("../../data/150_research_papers.csv")[['Link']]
#Add the label -> 1 for relevent
good_df['label'] = 1

In [3]:
good_df.head()
# good_df.shape

,Link,label
0,https://www.science.org/doi/10.1126/science.ad...,1
1,https://www.nature.com/articles/s41566-019-0398-2,1
2,https://www.nature.com/articles/s41560-020-007...,1
3,https://www.science.org/doi/10.1126/science.ab...,1
4,https://www.nature.com/articles/s41467-021-236...,1


## Import bad paper

In [4]:
bad_df = pd.read_csv("../../data/irrelevant_papers.csv")
bad_df.head()

,link
0,https://doi.org/10.1016/0021-9797(77)90150-3
1,https://www.sciencedirect.com/science/article/...
2,https://www.sciencedirect.com/science/article/...
3,https://doi.org/10.1107%2FS0021889813019535
4,https://www.sciencedirect.com/science/article/...


In [5]:
#Add the label -> 0 for irrelevent
bad_df['label'] = 0
# sampled_bad_df.to_csv('bad_paper_small.csv', index=False)

In [6]:
bad_df

,link,label
0,https://doi.org/10.1016/0021-9797(77)90150-3,0
1,https://www.sciencedirect.com/science/article/...,0
2,https://www.sciencedirect.com/science/article/...,0
3,https://doi.org/10.1107%2FS0021889813019535,0
4,https://www.sciencedirect.com/science/article/...,0
...,...,...
310,https://www.sciencedirect.com/science/article/...,0
311,https://www.sciencedirect.com/science/article/...,0
312,https://www.sciencedirect.com/science/article/...,0
313,https://doi.org/10.1063/1.1404988,0


# Scraping Text

In [7]:
download_dir = tempfile.mkdtemp()
chrome_options = uc.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,  # Set download location
    "download.prompt_for_download": False,       # Disable download prompts
    "plugins.always_open_pdf_externally": True   # Download PDFs instead of opening them
})
driver = uc.Chrome(options=chrome_options)

In [8]:
def get_text(soup):
    #Given a beautiful soup object, it will extact the text
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [9]:
def extract_text_from_urls(url, paper_index):

    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    text_extracted = get_text(soup)
    return text_extracted

In [10]:
#Record: took 7min 40sec
text_good_list = []
for index, row in good_df.iterrows():
    text_gotten = extract_text_from_urls(row['Link'], index)
    print(index, text_gotten[:30])
    text_good_list.append(text_gotten)

0 Improved charge extraction in 
1 Surface passivation of perovsk
2 Intact 2D/3D halide junction p
3 Deterministic fabrication of 3
4 Multication perovskite 2D/3D i
5 Front-contact passivation thro
6 Ligand-Stabilized Reduced-Dime
7 The 2D Halide Perovskite Ruleb
8 Reconfiguring the band-edge st
9 Defect Passivation of Organic–
10 Stability-limiting heterointer
11 3D Perovskite Passivation with
12 Record Open‐Circuit Voltage Wi
13 High‐Performance Flexible All‐
14 23.7% Efficient inverted perov
15 Imaging and quantifying non-ra
16 Deep surface passivation for e
17 Perovskite Quantum Wells Forma
18 Revealing the Transient Format
19 Engineering ligand reactivity 
20 How Stable Is the 2D/3D Interf
21 An interface stabilized perovs
22 Interfacial engineering from m
23 Quantum-size-tuned heterostruc
24 Accelerated aging of all-inorg
25 Thermal degradation of formami
26 On the Thermal and Thermodynam
27 Inorganic wide-bandgap perovsk
28 Thermal unequilibrium of strai
29 Amine additive reacti

In [ ]:
##EXTRACTED ALL 150 TEXT!!!
len(text_good_list)

151

In [21]:
#Store them as df and export
good_df['text'] = text_good_list
good_df = good_df.rename(columns={'Link': 'link'})
good_df.to_csv('../../data/text_good_paper.csv', index=False)

In [13]:
#Took 14 min
text_bad_list = []
for index, row in bad_df.iterrows():
    text_gotten = extract_text_from_urls(row['link'], index)
    print(index, len(text_gotten))
    text_bad_list.append(text_gotten)

0 3456
1 3360
2 124571
3 38426
4 3360
5 110
6 240304
7 3487
8 165506
9 9713
10 3357
11 59364
12 7900
13 3357
14 181061
15 3357
16 2909
17 84675
18 3357
19 122176
20 123830
21 36995
22 3360
23 42332
24 35463
25 2759
26 221399
27 47324
28 64653
29 3357
30 122176
31 124634
32 124594
33 98
34 3357
35 2130
36 63576
37 9455
38 26402
39 3357
40 90097
41 42061
42 124218
43 4581
44 87855
45 8055
46 3357
47 75571
48 176638
49 3360
50 9930
51 3357
52 1008237
53 3360
54 122176
55 40207
56 237238
57 3360
58 122176
59 99
60 114772
61 3360
62 252279
63 3357
64 122176
65 461631
66 58555
67 1070017
68 3360
69 2084
70 1869
71 52088
72 3357
73 122176
74 2976
75 158073
76 3360
77 122176
78 124634
79 124594
80 2421
81 3357
82 79548
83 3357
84 122176
85 161190
86 3357
87 122176
88 124634
89 149112
90 3360
91 121067
92 9976
93 198018
94 3357
95 50683
96 3357
97 4197
98 124592
99 124594
100 105897
101 61898
102 47550
103 72723
104 3357
105 122176
106 50217
107 160774
108 3360
109 50217
110 59740
111 61619
112

In [14]:
len(text_bad_list)

315

In [25]:
#Make sure to stop driver before you proceed
driver.quit()

In [15]:
bad_df['text'] = text_bad_list
bad_df.to_csv('../../data/text_bad_paper.csv', index=False)

In [16]:
good_df

,Link,label,text
0,https://www.science.org/doi/10.1126/science.ad...,1,Improved charge extraction in inverted perovsk...
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
3,https://www.science.org/doi/10.1126/science.ab...,1,Deterministic fabrication of 3D/2D perovskite ...
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
...,...,...,...
146,https://doi.org/10.1002%2Fsmll.201803350,1,General Nondestructive Passivation by 4‐Fluoro...
147,https://doi.org/10.1038%2Fs41467-022-34203-x,1,Overcoming C60-induced interfacial recombinati...
148,https://doi.org/10.1038%2Fs41560-023-01249-0,1,Bifunctional hole-shuttle molecule for improve...
149,https://doi.org/10.1002%2Fadma.201907757,1,Tailored Amphiphilic Molecular Mitigators for ...


In [17]:
bad_df

,link,label,text
0,https://doi.org/10.1016/0021-9797(77)90150-3,0,Steric stabilization - ScienceDirect\nJavaScri...
1,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
2,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
3,https://doi.org/10.1107%2FS0021889813019535,0,(IUCr) X-ray analysis of residual stress gradi...
4,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
...,...,...,...
310,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
311,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
312,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
313,https://doi.org/10.1063/1.1404988,0,The charge of glass and silica surfaces | The ...


In [22]:
merged_df = pd.concat([good_df, bad_df])
merged_df

,link,label,text
0,https://www.science.org/doi/10.1126/science.ad...,1,Improved charge extraction in inverted perovsk...
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
3,https://www.science.org/doi/10.1126/science.ab...,1,Deterministic fabrication of 3D/2D perovskite ...
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
...,...,...,...
310,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
311,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
312,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
313,https://doi.org/10.1063/1.1404988,0,The charge of glass and silica surfaces | The ...


In [31]:
merged_df.dropna(subset=['text'])

,link,label,text
0,https://www.science.org/doi/10.1126/science.ad...,1,Improved charge extraction in inverted perovsk...
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
3,https://www.science.org/doi/10.1126/science.ab...,1,Deterministic fabrication of 3D/2D perovskite ...
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
...,...,...,...
461,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
462,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
463,https://www.sciencedirect.com/science/article/...,0,The central role of colloids to explain the cr...
464,https://doi.org/10.1063/1.1404988,0,The charge of glass and silica surfaces | The ...


In [32]:
merged_df.to_csv('../../data/merged_label.csv', index=False)